Signor 3.0

In [11]:
targetGene = 'SLC25A20'

In [12]:
import http.client
import pandas as pd
from io import StringIO

# ids of Fatty Acid Biosynthesis
# ids = ['P53396','Q13085','P49327','P35610','O43772','P33121','Q9BW60','Q9NXB9','Q9HB03','Q9GZR5','Q9NYP7','A1L3X0','P37058',
# 'B0YJ81','Q9NZ01','O00767','Q9NR19']

ids = ['O43772']

final_df = pd.DataFrame(columns=['Database','TF','TargetGene','Library/PMID'])

for uid in ids:

    # Establish connection
    conn = http.client.HTTPSConnection("signor.uniroma2.it")

    # Define headers
    headersList = {
        "Accept": "*/*",
        "User-Agent": "Thunder Client (https://www.thunderclient.com)"
    }

    # Empty payload
    payload = ""

    # Make the request
    conn.request("GET", f"/getData.php?id={uid}&organism=9606", payload, headersList)

    # Get the response
    response = conn.getresponse()
    result = response.read()

    # Decode the response as a string
    decoded_result = result.decode("utf-8")

    # Convert the string data into a pandas DataFrame
    data = StringIO(decoded_result)
    df1 = pd.read_csv(data, sep='\t',header = None)

    if(len(df1) <= 1):
        continue

    print(df1)

    mask = (df1[9] == 'transcriptional regulation')

    for i in range(0,len(df1[mask])):
        new_row = {'Database' : 'Signor', 'TF' : list(df1[mask][0])[i],'TargetGene':list(df1[mask][4])[i],'Library/PMID':list(df1[mask][21])[0]}
        final_df = pd.concat([final_df, pd.DataFrame([new_row])],ignore_index = True)


      0        1       2        3         4        5       6        7   \
0  PPARD  protein  Q03181  UNIPROT  SLC25A20  protein  O43772  UNIPROT   
1  PPARA  protein  Q07869  UNIPROT  SLC25A20  protein  O43772  UNIPROT   

                                    8                           9   ...  19  \
0  up-regulates quantity by expression  transcriptional regulation  ... NaN   
1  up-regulates quantity by expression  transcriptional regulation  ... NaN   

   20        21  22  23      24  \
0 NaN  19577614   f NaN  miannu   
1 NaN  19577614   f NaN  miannu   

                                                  25             26     27  28  
0   CACT is upregulated by PPARalpha and PPARdelt...  SIGNOR-255048  0.309 NaN  
1   CACT is upregulated by PPARalpha and PPARdelt...  SIGNOR-255049  0.505 NaN  

[2 rows x 29 columns]


In [13]:
final_df

,Database,TF,TargetGene,Library/PMID
0,Signor,PPARD,SLC25A20,19577614
1,Signor,PPARA,SLC25A20,19577614



ChEA3

In [14]:
#### It returns all the TFs and a mapping to which query gene they belong to under the attribute Overlapping_Gene####

import requests
import json

# List of genes
genes = [targetGene,"SCD"]

# URL of the ChEA3 API endpoint
url = "https://maayanlab.cloud/chea3/api/enrich/"

# Payload to send with the POST request
payload = {
    "query_name": "myQuery",
    "gene_set": genes
}

# POST to ChEA3 server
response = requests.post(url, json=payload)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    results = response.json()
    # print(results)
    # If you want to pretty-print the results
    # print(json.dumps(results, indent=2))
else:
    print(f"Request failed with status code {response.status_code}")


# allTFs = response['Integrated--meanRank']


In [15]:
print(results.keys())

dict_keys(['Integrated--meanRank', 'Integrated--topRank', 'GTEx--Coexpression', 'ReMap--ChIP-seq', 'Enrichr--Queries', 'ENCODE--ChIP-seq', 'ARCHS4--Coexpression', 'Literature--ChIP-seq'])


In [16]:
results['Literature--ChIP-seq']

[{'Query Name': 'myQuery',
  'Rank': '1',
  'Scaled Rank': '0.006098',
  'Set_name': 'HNF4A_19761587_CHIPCHIP_CACO2_HUMAN',
  'TF': 'HNF4A',
  'Intersect': '2',
  'Set length': '1033',
  'FET p-value': '0.01353',
  'FDR': '1.0',
  'Odds Ratio': '19.4',
  'Library': 'Literature--ChIP-seq',
  'Overlapping_Genes': 'SCD,SLC25A20'},
 {'Query Name': 'myQuery',
  'Rank': '2',
  'Scaled Rank': '0.0122',
  'Set_name': 'VDR_20736230_CHIPSEQ_LYMPHOBLASTOID_HUMAN',
  'TF': 'VDR',
  'Intersect': '1',
  'Set length': '205',
  'FET p-value': '0.03013',
  'FDR': '1.0',
  'Odds Ratio': '49.01',
  'Library': 'Literature--ChIP-seq',
  'Overlapping_Genes': 'SLC25A20'},
 {'Query Name': 'myQuery',
  'Rank': '3',
  'Scaled Rank': '0.01829',
  'Set_name': 'CDX2_19796622_CHIPSEQ_MESC_MOUSE',
  'TF': 'CDX2',
  'Intersect': '1',
  'Set length': '275',
  'FET p-value': '0.04014',
  'FDR': '1.0',
  'Odds Ratio': '36.49',
  'Library': 'Literature--ChIP-seq',
  'Overlapping_Genes': 'SLC25A20'},
 {'Query Name': 'myQu

In [17]:
allTFs = results['Literature--ChIP-seq']
# print(json.dumps(allTFs,indent=2))

for dump in allTFs:
    overLappingGenes = dump['Overlapping_Genes'].split(',')
    if(targetGene in overLappingGenes):
        new_row = {'Database' : 'CHEA', 'TF' : dump["TF"],'TargetGene':targetGene,'Library/PMID':dump["Library"]}
        final_df = pd.concat([final_df, pd.DataFrame([new_row])],ignore_index = True)

In [18]:
final_df

,Database,TF,TargetGene,Library/PMID
0,Signor,PPARD,SLC25A20,19577614
1,Signor,PPARA,SLC25A20,19577614
2,CHEA,HNF4A,SLC25A20,Literature--ChIP-seq
3,CHEA,VDR,SLC25A20,Literature--ChIP-seq
4,CHEA,CDX2,SLC25A20,Literature--ChIP-seq
5,CHEA,PPARD,SLC25A20,Literature--ChIP-seq
6,CHEA,NANOG,SLC25A20,Literature--ChIP-seq
7,CHEA,ERG,SLC25A20,Literature--ChIP-seq
8,CHEA,TCF3,SLC25A20,Literature--ChIP-seq
9,CHEA,SRF,SLC25A20,Literature--ChIP-seq


trrust

In [19]:
import pandas as pd

df2 = pd.read_csv('D:\\Raylab\\LiMeNEx\\FetchData\\trrust_rawdata.human.tsv',sep = '\t' ,header= None)
df2.columns = ['TF','Target','Regulation','PMID']

df2.head()

df2 = df2[(df2["Target"] == targetGene)]

for i in range(0,len(df2)):
    new_row = {'Database' : 'TRRUST', 'TF' : list(df2['TF'])[i],'TargetGene':targetGene,'Library/PMID':list(df2["PMID"])[i]}
    final_df = pd.concat([final_df, pd.DataFrame([new_row])],ignore_index = True)


In [20]:
final_df

,Database,TF,TargetGene,Library/PMID
0,Signor,PPARD,SLC25A20,19577614
1,Signor,PPARA,SLC25A20,19577614
2,CHEA,HNF4A,SLC25A20,Literature--ChIP-seq
3,CHEA,VDR,SLC25A20,Literature--ChIP-seq
4,CHEA,CDX2,SLC25A20,Literature--ChIP-seq
5,CHEA,PPARD,SLC25A20,Literature--ChIP-seq
6,CHEA,NANOG,SLC25A20,Literature--ChIP-seq
7,CHEA,ERG,SLC25A20,Literature--ChIP-seq
8,CHEA,TCF3,SLC25A20,Literature--ChIP-seq
9,CHEA,SRF,SLC25A20,Literature--ChIP-seq


Save CSV file

In [98]:
final_df.to_csv(f"D:/Raylab/LiMeNEx/FetchData/{targetGene}.csv")